# Практика 8: Методы защиты от атак на модели ИИ
Выполнил студент группы ББМО-01-23 Стрыбак Вячелсав

Шаг 1: Защита с помощью Adversarial Training

Adversarial Training — это метод защиты, который заключается в том, чтобы обучать модель на противоречивых примерах. Этот метод помогает модели научиться быть более устойчивой к атакам, так как она сталкивается с противоречивыми примерами на этапе обучения.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Уменьшаем количество обучающих изображений до 1000
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)  # Ограничение значений пикселей
    return adversarial_image

# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)  # Преобразование в tf.Tensor
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)  # Преобразование метки в Tensor

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())  # Конвертируем обратно в numpy
        adversarial_images.append(adversarial_image.reshape(28, 28))  # Преобразуем в форму (28, 28)

    return np.array(adversarial_images)

# Создание модели
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5):  # Короткое обучение для демонстрации
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i+32]
            batch_labels = train_labels[i:i+32]
            # Генерация противоречивых примеров для текущей партии данных
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            # Объединение оригинальных и противоречивых примеров
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            # Обучение на комбинированных данных
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Шаг 2: Градиентная маскировка (Gradient Masking)

Gradient Masking — это метод защиты, который затрудняет доступ к градиентам модели для атак. Он
используется для уменьшения информации, доступной для атакующих, и усложнения поиска
направленных изменений.

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Загрузка и подготовка данных
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Обновление модели для градиентной маскировки
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')  # Используем softplus вместо softmax для градиентной маскировки
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8714 - loss: 0.4650
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9606 - loss: 0.1334
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9742 - loss: 0.0863
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9796 - loss: 0.0681
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9846 - loss: 0.0509


Шаг 3: Регуляризация и нормализация для повышения устойчивости

In [3]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model

# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.7905 - loss: 1.3430
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8676 - loss: 0.6251
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8646 - loss: 0.6203
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8633 - loss: 0.6084
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8614 - loss: 0.6131


Шаг 4: Оценка моделей на противоречивых примерах

In [5]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')

# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model,
test_images, test_labels, epsilon=0.1)

# Оценка защищенной модели
test_loss, test_acc = protected_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

# Оценка модели с градиентной маскировкой
test_loss, test_acc = masked_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

# Оценка модели с регуляризацией и нормализацией
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of regularized model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4850 - loss: 1.4720
Accuracy of protected model on adversarial examples: 0.5443999767303467
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6525 - loss: 1.2603
Accuracy of masked model on adversarial examples: 0.6858000159263611
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7063 - loss: 1.0677
Accuracy of regularized model on adversarial examples: 0.7394000291824341


Вывод:


Ожидалось, что методы защиты, такие как Adversarial Training и Gradient Masking, увеличат точность
моделей на противоречивых примерах. Регуляризация и нормализация могут дополнительно
помочь в повышении устойчивости модели к атакам.  
  Защищенная модель показала точность 48% и уровень потерь 1.4. Это неплохой результат, но все же его можно улучшить.  
  Модель с градиентной маскировкой показывает точность 68% и уровень потерь 1.2. Этот результат доказывает, что такая модель более устойчива к атакам на противоречивых примерах, чем обычная защищенная.  
Однако, наиболее результативной оказалось модель с регуляризацией и нормализацией, которая показывает точность 70% и уровень потерь 1. Такая модель, при её доработке или комбинировании методов защиты может значительно увеличить свою устойчивость атакам и повысить оцениваемые метрики.